### Import necessary libraryies

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 122kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 710kB/s eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-cp37-none-any.whl size=76240 sha256=60a9331215a6acbc6ab919c6c7a5abdca555ba92651f9923ed34c0d2c8b3fc1e
  Stored in directory: /Users/justinwu/Library/Caches/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
Folium installed
Libraries imported.


### Foursqure setting

In [39]:
CLIENT_ID = 'FGF1SK2HYOHKUIPSVAKXSEPTVYRRS3FMS4OE4WTWKDDYSHTE' # your Foursquare ID
CLIENT_SECRET = '1FDO4E41CFGZBFYRKGE1TOXR2NM5DD0MIFNCC2GY5RHH5SS4' # your Foursquare Secret
ACCESS_TOKEN = 'UYPUGMAYB2TFJACUJN3BAM5ERGZZUHW3B1NQTH0LL12K2RN2' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 3000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FGF1SK2HYOHKUIPSVAKXSEPTVYRRS3FMS4OE4WTWKDDYSHTE
CLIENT_SECRET:1FDO4E41CFGZBFYRKGE1TOXR2NM5DD0MIFNCC2GY5RHH5SS4


In [67]:
Taipei_address = "Taipei 101"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(Taipei_address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

25.03368625 121.56481033101315


In [74]:
search_query = 'bubble tea shop'
radius = 40000
print(search_query + ' .... OK!')

bubble tea shop .... OK!


In [75]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [76]:
results = requests.get(url).json()

In [77]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.neighborhood,location.crossStreet
0,5d88c3d6422f4000087cf201,老賴茶棧 Like Tea Shop 松山店,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",v-1627792180,False,25.047748,121.557859,"[{'label': 'display', 'lat': 25.047748, 'lng':...",1715,10563,TW,松山區,臺北市,臺灣,"[松山區, 臺北市 10563]",NaN,NaN,NaN
1,51dbf114498e9309cd20ba09,波哥茶飲 Bog Tea Shop,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",v-1627792180,False,25.033859,121.543596,"[{'label': 'display', 'lat': 25.03385852679692...",2139,NaN,TW,台北市,NaN,臺灣,"[復興南路一段321號, 台北市]",復興南路一段321號,NaN,NaN
2,57233abf498e31b014397288,紅太陽茶飲 Red Sun Tea Shop 公館台大店,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",v-1627792180,False,25.015122,121.532630,"[{'label': 'display', 'lat': 25.015122, 'lng':...",3848,NaN,TW,NaN,NaN,臺灣,NaN,NaN,NaN,NaN
3,5cb47a51f4b5250039e0b54b,mita tea shop 米塔黑糖,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",v-1627792180,False,25.029719,121.554107,"[{'label': 'display', 'lat': 25.029719, 'lng':...",1166,106,TW,Da-an District,NaN,臺灣,"[No. 103, Tung-hua Street, Da-an District, 106]","No. 103, Tung-hua Street",NaN,NaN
4,5ce90ab700b068002daaa7ec,Wanpo Tea Shop (萬波島嶼紅茶),"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",v-1627792180,False,25.042366,121.512879,"[{'label': 'display', 'lat': 25.04236589218256...",5326,100,TW,中正區,臺北市,臺灣,"[衡陽路25號, 中正區, 臺北市 100]",衡陽路25號,NaN,NaN


In [78]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,neighborhood,crossStreet,id
0,老賴茶棧 Like Tea Shop 松山店,Bubble Tea Shop,25.047748,121.557859,"[{'label': 'display', 'lat': 25.047748, 'lng':...",1715,10563,TW,松山區,臺北市,臺灣,"[松山區, 臺北市 10563]",NaN,NaN,NaN,5d88c3d6422f4000087cf201
1,波哥茶飲 Bog Tea Shop,Tea Room,25.033859,121.543596,"[{'label': 'display', 'lat': 25.03385852679692...",2139,NaN,TW,台北市,NaN,臺灣,"[復興南路一段321號, 台北市]",復興南路一段321號,NaN,NaN,51dbf114498e9309cd20ba09
2,紅太陽茶飲 Red Sun Tea Shop 公館台大店,Bubble Tea Shop,25.015122,121.532630,"[{'label': 'display', 'lat': 25.015122, 'lng':...",3848,NaN,TW,NaN,NaN,臺灣,NaN,NaN,NaN,NaN,57233abf498e31b014397288
3,mita tea shop 米塔黑糖,Bubble Tea Shop,25.029719,121.554107,"[{'label': 'display', 'lat': 25.029719, 'lng':...",1166,106,TW,Da-an District,NaN,臺灣,"[No. 103, Tung-hua Street, Da-an District, 106]","No. 103, Tung-hua Street",NaN,NaN,5cb47a51f4b5250039e0b54b
4,Wanpo Tea Shop (萬波島嶼紅茶),Bubble Tea Shop,25.042366,121.512879,"[{'label': 'display', 'lat': 25.04236589218256...",5326,100,TW,中正區,臺北市,臺灣,"[衡陽路25號, 中正區, 臺北市 100]",衡陽路25號,NaN,NaN,5ce90ab700b068002daaa7ec


In [79]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
    
venues_map